In [53]:
import pandas as pd
import datetime
from datetime import datetime
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

%matplotlib inline

In [54]:

df = pd.read_csv("sphist1.csv")
df["Date"] = pd.to_datetime(df["Date"])
df = df.sort_values("Date")
df = df.reset_index(drop=True)
print(df.columns)

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close'], dtype='object')


In [55]:
df["Date in word"]=df["Date"].apply( lambda date: date.strftime("%b %d, %Y"))
df.head()

,Date,Open,High,Low,Close,Volume,Adj Close,Date in word
0,1950-01-03,16.66,16.66,16.66,16.66,1260000.0,16.66,"Jan 03, 1950"
1,1950-01-04,16.85,16.85,16.85,16.85,1890000.0,16.85,"Jan 04, 1950"
2,1950-01-05,16.93,16.93,16.93,16.93,2550000.0,16.93,"Jan 05, 1950"
3,1950-01-06,16.98,16.98,16.98,16.98,2010000.0,16.98,"Jan 06, 1950"
4,1950-01-09,17.08,17.08,17.08,17.08,2520000.0,17.08,"Jan 09, 1950"


## Making Feature Columns

In [56]:
Past_5_ave_list = []
Past_5_std_list = []
Past_5_ave_vol_list = []
Past_5_std_vol_list=[]
Past_30_ave_list = []
Past_30_std_list = []
Past_30_ave_vol_list = []

for index, row in df.iterrows():
    if index < 5:
        Past_5_ave_list.append(0)
        Past_5_std_list.append(0)
        Past_5_ave_vol_list.append(0)
        Past_5_std_vol_list.append(0)
    if index < 30:
        Past_30_ave_list.append(0)
        Past_30_std_list.append(0)
        Past_30_ave_vol_list.append(0)
    if index >= 5:
        i_5 = index - 5
        last_5_ave = df["Close"].iloc[i_5:index].mean()
        last_5_std = df["Close"].iloc[i_5:index].std()
        last_5_ave_vol = df["Volume"].iloc[i_5:index].mean()
        last_5_ave_vol = df["Volume"].iloc[i_5:index].std()
        Past_5_ave_list.append(last_5_ave)
        Past_5_std_list.append(last_5_std)
        Past_5_ave_vol_list.append(last_5_ave_vol)
        Past_5_std_vol_list.append( last_5_ave_vol)
    if index >= 30:
        i_30 = index - 30
        last_30_ave = df["Close"].iloc[i_30:index].mean()
        Past_30_ave_list.append(last_30_ave)
        Past_30_std = df["Close"].iloc[i_30:index].std()
        Past_30_std_list.append(Past_30_std)
        last_30_ave_vol = df["Volume"].iloc[i_30:index].mean()
        Past_30_ave_vol_list.append(last_30_ave_vol)

df["Past 5 Ave"] = Past_5_ave_list
df["Past 30 Ave"] = Past_30_ave_list
df["Past 5 Std"] = Past_5_std_list
df["Past 30 Std"] = Past_30_std_list
df["Past 5 Ave Volume"] = Past_5_ave_vol_list
df["Past 30 Ave Volume"] = Past_30_ave_list
df["Past 5 std Volume"] = Past_5_std_vol_list



In [57]:
# Creating new columns for the standard deviation of the last 5 days of the "Past 5 Ave" column
Past_5_ave_std_list = []
for index, row in df.iterrows():
    if index < 5:
        Past_5_ave_std_list.append(0)
    if index >= 5:
        i_5 = index - 5
        last_5_ave_std = df["Past 5 Ave"].iloc[i_5:index].std()
        Past_5_ave_std_list.append(last_5_ave_std)
df["Past 5 Ave Std"] = Past_5_ave_std_list

In [58]:
# # Creating new columns for previous 365 day "Close" (mean, standard deviation, min and max), "Volume" (mean)
Past_365_ave_list = []
Past_365_std_list = []
Past_365_ave_vol_list = []
Past_365_std_vol_list = []
Past_365_max_list = []
Past_365_min_list = []

for index, row in df.iterrows():
    if index < 365:
        Past_365_ave_list.append(0)
        Past_365_std_list.append(0)
        Past_365_ave_vol_list.append(0)
        Past_365_std_vol_list.append(0)
        Past_365_max_list.append(0)
        Past_365_min_list.append(0)
    if index >= 365:
        i_365 = index - 365
        last_365_ave = df["Close"].iloc[i_365:index].mean()
        last_365_std = df["Close"].iloc[i_365:index].std()
        last_365_ave_vol = df["Volume"].iloc[i_365:index].mean()
        last_365_std_vol = df["Volume"].iloc[i_365:index].std()
        Past_365_max = df["Close"].iloc[i_365:index].max()
        Past_365_min = df["Close"].iloc[i_365:index].min()
        Past_365_ave_list.append(last_365_ave)
        Past_365_std_list.append(last_365_std)
        Past_365_ave_vol_list.append(last_365_ave_vol)
        Past_365_std_vol_list.append(last_365_std_vol)
        Past_365_max_list.append(Past_365_max)
        Past_365_min_list.append(Past_365_min)

df["Past 365 Ave"] = Past_365_ave_list
df["Past 365 Std"] = Past_365_std_list
df["Past 365 Ave Volume"] = Past_365_ave_vol_list
df["Past 365 Max"] = Past_365_max_list
df["Past 365 Min"] = Past_365_min_list
df["Past 365 Std Volume"]=Past_365_std_vol_list

In [59]:
# Creating columns for ratio of previous days "Close" vs. "Past 365 Min" and "Past 365 Max"  
price_365_min_ratio_list = []
price_365_max_ratio_list = []
for index, row in df.iterrows():
    if index < 366:
        price_365_min_ratio_list.append(0)
        price_365_max_ratio_list.append(0)
    if index >= 366:
        i_1 = index - 1
        price_365_min_ratio = df["Close"].iloc[i_1] / df["Past 365 Min"].iloc[index]
        price_365_max_ratio = df["Close"].iloc[i_1] / df["Past 365 Max"].iloc[index]
        price_365_min_ratio_list.append(price_365_min_ratio)
        price_365_max_ratio_list.append(price_365_max_ratio)
df["Price 365 Max Ratio"] = price_365_min_ratio_list
df["Price 365 Min Ratio"] = price_365_max_ratio_list

In [60]:
df = df.reset_index(drop=True)


In [61]:
#The ratio between the average volume for the past five days, and the average volume for the past year.

price_365_avg_vol_ratio_list = []

for index, row in df.iterrows():
    if df["Past 365 Ave Volume"].iloc[index]!=0:
        price_365_avg_vol_ratio_list.append( df["Past 5 Ave Volume"].iloc[index] / df["Past 365 Ave Volume"].iloc[index] )
    else:
        price_365_avg_vol_ratio_list.append(0)
        
        
df["Price 365 avg vol ratio"] = price_365_avg_vol_ratio_list


In [62]:
# Eliminating rows with 0 in new columns and NA values
df = df[ df["Date"] > datetime(year=1951, month=1, day=2)]
df = df.dropna()

In [63]:
# Splitting dataframe into train and test sets
train = df[df["Date"] < datetime(year=2013, month=1, day=1)]
test = df[df["Date"] >= datetime(year=2013, month=1, day=1)]

In [ ]:
# Training a Linear Regression model

lr = LinearRegression()
cols = ['Past 5 Ave', 'Past 30 Ave', 'Past 5 Std',
       'Past 30 Std', 'Past 5 Ave Volume', 'Past 30 Ave Volume',
       'Past 5 std Volume', 'Past 5 Ave Std', 'Past 365 Ave', 'Past 365 Std',
       'Past 365 Ave Volume', 'Past 365 Max', 'Past 365 Min',
       'Past 365 Std Volume', 'Price 365 Max Ratio', 'Price 365 Min Ratio',
       'Price 365 avg vol ratio']
lr.fit(train[cols], train["Close"])

# Using the Linear Regression model to make predictions and determining error metric
predictions = lr.predict(test[cols])


mae = mean_absolute_error(test["Close"], predictions)
mse = mean_squared_error(test["Close"], predictions)

print("MAE : %f  MSE : %f" %(mae,mse))
print(test["Close"].mean())


We use Mean Absolute Error, also called MAE, as an error metric, because it will show you how "close" you were to the price in intuitive terms. Mean Squared Error, or MSE, is an alternative that is more commonly used, but makes it harder to intuitively tell how far off you are from the true price because it squares the error.

Our Mean Absolute Error is off by just over 15.91 points and our average closing price is just under 1875. This isn't a model we would want to use to actually make stock trades, but it isn't bad considering we only used features derived from the closing price and volume history.

In [ ]:
close = test["Close"]
date  = test["Date"]
fig = plt.figure(figsize=(10,5))
plt.plot(date, close, c='red')
plt.plot(date, predictions, c='blue')
plt.xticks(rotation=45)
plt.xlabel("Date")
plt.ylabel("Closing Price")
plt.title("Actual vs. Predicted Closing Prices")
plt.show()

## Lets try improving our model

Accuracy would improve greatly by making predictions only one day ahead. For example, train a model using data from 1951-01-03 to 2013-01-02, make predictions for 2013-01-03, and then train another model using data from 1951-01-03 to 2013-01-03, make predictions for 2013-01-04, and so on. This more closely simulates what you'd do if you were trading using the algorithm.

In [66]:
np.where(df["Date"] == datetime(year=2013, month=1, day=3))

(array([15602]),)

In [67]:
cols = ['Past 5 Ave', 'Past 30 Ave', 'Past 5 Std',
       'Past 30 Std', 'Past 5 Ave Volume', 'Past 30 Ave Volume',
       'Past 5 std Volume', 'Past 5 Ave Std', 'Past 365 Ave', 'Past 365 Std',
       'Past 365 Ave Volume', 'Past 365 Max', 'Past 365 Min',
       'Past 365 Std Volume', 'Price 365 Max Ratio', 'Price 365 Min Ratio',
       'Price 365 avg vol ratio']

In [68]:
# Training a Linear Regression model


lr1 = LinearRegression()
df["Predicted Close"]=0
predictions=[]
mae_list=[]
close=[]
predicted_close=[]
for index,row in df.iterrows():
    
    # Splitting dataframe into train and test sets
    if index>15602:
        train = df.iloc[:index]
        test  = df.iloc[index]
        close.append(test["Close"])
        test = test[:, np.newaxis]
        lr1.fit(train[cols], train["Close"])
        predict = lr1.predict(test[8:25,0].reshape(1,-1))
        predictions.append(predict[0])
        predicted_close.append(predict[0])
        
    else :
        predictions.append(0)

df["Predicted Close"]=predictions
mae=mean_absolute_error(close,predicted_close)
print(mae)

IndexError: single positional indexer is out-of-bounds

to do :
-  You can also improve the algorithm used significantly. Try other techniques, like a random forest, and see if they perform better.

-  You can also incorporate outside data, such as the weather in New York City (where most trading happens) the day before, and the amount of Twitter activity around certain stocks.

-  You can also make the system real-time by writing an automated script to download the latest data when the market closes, and make predictions for the next day.

-  Finally, you can make the system "higher-resolution". You're currently making daily predictions, but you could make hourly, minute-by-minute, or second by second predictions. This will require obtaining more data, though. You could also make predictions for individual stocks instead of the S&P500.

In [ ]:
test  = df.iloc[15602:]
plt.scatter(test["Close"], test["Predicted Close"])
plt.plot(test["Close"], test["Predicted Close"])
plt.show()